In [1]:
import networkx as nx
import straky

In [2]:
N = 4
board = straky.Board(N)

In [3]:
board.play_game(players=[straky.player.PlayerRandom(), straky.player.PlayerRandom()])

New game.

Player 1 pushes row 4 left with tile ░░.
  A B C D 
1 ________
2 ________
3 ________
4 ______░░

Player 2 places tile ██ at B1.
  A B C D 
1 __██____
2 ________
3 ________
4 ______░░

Player 1 pushes column B up with tile ░░.
  A B C D 
1 ________
2 ________
3 ________
4 __░░__░░

Player 2 places tile ██ at A3.
  A B C D 
1 ________
2 ________
3 ██______
4 __░░__░░

Player 1 pushes row 4 left with tile ░░.
  A B C D 
1 ________
2 ________
3 ██______
4 ░░__░░░░

Player 2 places tile ██ at A1.
  A B C D 
1 ██______
2 ________
3 ██______
4 ░░__░░░░

Player 1 places tile ░░ at D2.
  A B C D 
1 ██______
2 ______░░
3 ██______
4 ░░__░░░░

Player 2 flips row 1.
  A B C D 
1 ░░______
2 ______░░
3 ██______
4 ░░__░░░░

Player 1 places tile ░░ at B4.
  A B C D 
1 ░░______
2 ______░░
3 ██______
4 ░░░░░░░░

Player 1 wins!
